# Rating Prediction 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk

In [ ]:
df = pd.read_csv("Musical_instruments_reviews.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.columns

EDA

In [ ]:
df.isnull().sum()

filling missing values

In [ ]:
df['reviewText'].fillna("", inplace = True)

In [ ]:
df['reviewText'].isnull().sum()

In [ ]:
df['review'] = df['reviewText']+df['summary']

In [ ]:
df['review'][0]

In [ ]:
df.head()

Dropping 

In [ ]:
df = df.drop(['reviewerID','asin','helpful','reviewText','reviewerName','unixReviewTime','reviewTime','summary'],axis=1)

In [ ]:
df.columns

In [ ]:
df['overall'].value_counts()

In [ ]:
def rating(rating):
    if(int(rating) == 1 or int(rating) == 2 or int(rating) == 3):
        return 0
    else: 
        return 1
df.overall = df.overall.apply(rating) 

In [ ]:
df['overall'].value_counts()

Cleaning , tokenizing

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
import re


In [ ]:
import nltk
nltk.download('stopwords')

removing punctuation

In [ ]:
import string
from string import punctuation

In [ ]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [ ]:
from nltk.corpus import wordnet

In [ ]:
def get_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

lemmatizer

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
from nltk import pos_tag

In [ ]:
def lemmatize_words(review):
    final_review = []
    for i in review.split():
        if i.strip().lower() not in stop:
            pos = pos_tag([i.strip()])
            word = lemmatizer.lemmatize(i.strip(),get_pos(pos[0][1]))
            final_review.append(word.lower())
    return " ".join(final_review)

In [ ]:
df['review'] = df.review.apply(lemmatize_words)

In [ ]:
df['review']

model fitting & prediction

In [ ]:
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df.review,df.overall,test_size = 0.2 , random_state = 0)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(min_df= 0,max_df= 1,binary=False,ngram_range= (1,3))

cv_train_reviews = cv.fit_transform(x_train)

cv_test_reviews = cv.transform(x_test)

print('cv_train:',cv_train_reviews.shape)
print('cv_test:',cv_test_reviews.shape)

LogisticRegression

In [ ]:
lr = LogisticRegression(penalty= 'l2',max_iter=500,C = 1,random_state= 0)
 
lr_bow = lr.fit(cv_train_reviews,y_train)
print(lr_bow)

lr_tfidf = lr.fit(cv_train_reviews,y_train)
print(lr_tfidf)

In [ ]:
lr_bow_predict = lr.predict(cv_test_reviews)

lr_tfidf_predict = lr.predict(cv_test_reviews)

In [ ]:
lr_bow_score = accuracy_score(y_test,lr_bow_predict)
print("lr_bow_score :",lr_bow_score)
#Accuracy score
lr_tfidf_score = accuracy_score(y_test,lr_tfidf_predict)
print("lr_tfidf_score :",lr_tfidf_score)

classification_report

In [ ]:
lr_bow_report = classification_report(y_test,lr_bow_predict,target_names=['0','1'])
print(lr_bow_report)

#Classification report 
lr_tfidf_report = classification_report(y_test,lr_tfidf_predict,target_names=['0','1'])
print(lr_tfidf_report)

naive bayes MultinomialNB

In [ ]:
mnb = MultinomialNB()

mnb_bow = mnb.fit(cv_train_reviews,y_train)
print(mnb_bow)

mnb_tfidf = mnb.fit(cv_train_reviews,y_train)
print(mnb_tfidf)

In [ ]:
mnb_bow_predict = mnb.predict(cv_test_reviews)

mnb_tfidf_predict = mnb.predict(cv_test_reviews)

In [ ]:
mnb_bow_score = accuracy_score(y_test,mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)
mnb_tfidf_score = accuracy_score(y_test,mnb_tfidf_predict)
print("mnb_tfidf_score :",mnb_tfidf_score)

classification_report :naive bayes MultinomialNB

In [ ]:
mnb_bow_report = classification_report(y_test,mnb_bow_predict,target_names = ['0','1'])
print(mnb_bow_report)
mnb_tfidf_report = classification_report(y_test,mnb_tfidf_predict,target_names = ['0','1'])
print(mnb_tfidf_report)

SVM 

In [ ]:
ls =  SVC()
ls_bow = ls.fit(cv_train_reviews,y_train)
ls_tfidf = ls.fit(cv_train_reviews,y_train)

#predict
ls_bow_predict = ls.predict(cv_test_reviews)
ls_tfidf_predict = ls.predict(cv_test_reviews)

#accuracy
ls_bow_score = accuracy_score(y_test, ls_bow_predict)
ls_tfidf_score = accuracy_score(y_test,ls_tfidf_predict)

print('ls bow accuracy:', ls_bow_score)
print('ls tfidf accuracy:', ls_tfidf_score)

classification_report SVM

In [ ]:
ls_bow_report = classification_report(y_test,ls_bow_predict,target_names = ['0','1'])
print(ls_bow_report)
ls_tfidf_report = classification_report(y_test,ls_tfidf_predict,target_names = ['0','1'])
print(ls_tfidf_report)

In [ ]:
df['review'][3]

In [ ]:
mnb_bow_predict[3]

rating 1,2,3 = 0
rating 4,5 = 1

In [ ]:

from sklearn.ensemble import VotingClassifier
lr = LogisticRegression( random_state=0)
mnb = MultinomialNB()
ls =  SVC()
model = VotingClassifier(estimators=[('lr', lr), ('mnb', mnb), ('ls', ls)], voting='hard')
model.fit(X_train,Y_train)
model.score(X_test,Y_test)